In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2019-07-02 05:00:00+00:00,136.981,137.088,136.482,136.546
1,2019-07-02 09:00:00+00:00,136.548,136.817,136.460,136.621
2,2019-07-02 13:00:00+00:00,136.619,136.766,135.904,136.089
3,2019-07-02 17:00:00+00:00,136.087,136.087,135.866,135.886
4,2019-07-02 21:00:00+00:00,135.878,135.934,135.580,135.603


In [6]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [7]:
df_plot = df_ma.iloc[:500]

In [8]:
cp = CandlePlot(df_plot)

In [9]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [10]:
cp.show_plot(line_traces=traces)

In [ ]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [ ]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [ ]:
df_an.head()

In [ ]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [ ]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [ ]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [ ]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [ ]:
df_trades.head()

In [ ]:
cp = CandlePlot(df_an.iloc[220:260])
cp.show_plot(line_traces=[MA_S, MA_L])

In [ ]:
ic.LoadInstruments("../data")

In [ ]:
ic.instruments_dict[pair]

In [ ]:
ins_data = ic.instruments_dict[pair]

In [ ]:
df_trades.shape

In [ ]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [ ]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [ ]:
df_trades.GAIN.sum()

In [ ]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [ ]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])